# Conditional Probability Queries

### A. Problem Description

* **a. Describing Problem**

    * Evidence: $E=e$
    * Query: A subset of variables $Y$
    * Task: Compute $P(Y|E=e)$
    
* **b. Procedure**

    * Expressing computation task
        * $ P(Y|E=e) = \frac{P(Y,E=e)}{P(E=e)} $
    * Numerator    
        * $P(Y,E=e) = \sum_WP(Y,W,E=e) = \sum_W\frac{1}{Z}\prod_k\phi_k(D_k,E=e)$, where $W=\{X_1,...,X_n\} - Y - E$ (i.e. nonquery and nonevidence variables).
        * Renormalization: $P(Y,E=e) = ... \propto \sum_W\prod_k\phi_k^{'}(D_k^{'})$ then renormalizing by computing $Z$ (note, $\phi_k^{'}(D_k^{'})$ are E!=e eliminated reduced factors).
        * NB: summing-out is usually intractable, if the number of variables is large.
    * Denominator
        * $P(E=e) = \sum_Y\sum_W\frac{1}{Z}\phi_k^{'}(D_k^{'})$

### B. General Algorithms

* **a. Variable Elimination** (EXACT)

* **b. Belief Propagation & Variational Approximations** (EXACT, APPROX)

* **c. Random Sampling Instantiations (MCMC, Importance Sampling)** (APPROX)

### C. Maximum a Posteriori (MAP)

* **a. Describing Task**

    * Evidence: $E=e$
    * Query: All the rest of the variables $Y$, for $Y=\{X_1,...,X_n\}-E$
    * Task: Compute $MAP(Y|E=e) = argmax_yP(Y=y|E=e)$ (i.e. given $E=e$, assignments of $Y$s that maximizes the probability $P(Y|E=e)$)
    * NB: there may be more than 1 possible solution.

* **b. Application E.G.**

    * Message Decoding: most likely message given some pieces.
    * Image Segmentation: most liely segmentation (i.e. assignments of pixels to classes).
    
* **c. Max-Product Approach**

    * $ P(Y|E=e) = \frac{P(Y,E=e)}{P(E=e)} \propto P(Y,E=e)$
    * $ P(Y,E=e) = \frac{1}{Z}\prod_k\phi_k^{'}(D_k^{'}) \propto \prod_k\phi_k^{'}(D_k^{'}) $, where the product is of the reduced factors wrt. the evidence.
    * $ argmax_YP(Y|E=e) = argmax_Y\prod_k\phi_k^{'}(D_k^{'}) $

# Variable Elimination

**Objective: ** Efficiently compute some joint/conditional probability.

### A. Chain VE

In [1]:
#
#     A --- B --- C --- D --- E 
#      

** No evidence **

** Query: ** $ P(E) $ 

$ P(E) \propto \sum_D\sum_C\sum_B\sum_A \hat{P}(A,B,C,D,E) $

$ = \sum_D\sum_C\sum_B\sum_A \phi_1(A,B)\phi_2(B,C)\phi_3(C,D)\phi_4(D,E) $ <= Factorization

$ = \sum_D\sum_C\sum_B \phi_2(B,C)\phi_3(C,D)\phi_4(D,E) \sum_A \phi_1(A,B) $ <= Eliminate A

$ = \sum_D\sum_C\sum_B \phi_2(B,C)\phi_3(C,D)\phi_4(D,E)\tau_1(B) $

$ = \sum_D\sum_C \phi_3(C,D)\phi_4(D,E) \sum_B \phi_2(B,C)\tau_1(B) $ <= Eliminate B

$ = \sum_D\sum_C \phi_3(C,D)\phi_4(D,E)\tau_2(C) $

$ = \sum_D \phi_4(D,E) \sum_C \phi_3(C,D)\tau_2(C) $ <= Eliminate C

$ = \sum_D \phi_4(D,E)\tau_3(D) $ <= Eliminate D

$ = \tau_4(E) $ <= Compute P(E)

### B. Non-Chain VE 

In [ ]:
#   C
#   |_
#   D     I
#    \_ /_ \_
#     G    S
#    / |_  |
#   /  L   |
#  /_   \_ |_
# H |_____J

** a. No Evidence **

** Query: ** $P(J)$

$ P(J) \propto \sum_{L,S,G,H,I,D,C} \phi_J(J,L,S)\phi_L(L,G)\phi_S(S,I)\phi_G(G,I,D)\phi_H(H,G,J)\phi_I(I)\phi_D(C,D)\phi_C(C) $

$ = ... $  <= Elimination Order: $C\rightarrow D\rightarrow I\rightarrow H\rightarrow G\rightarrow S\rightarrow L$

$ = \tau_k(J) $

** b. Evidence **

** Query 1 (Joint): ** $P(J,I=i,H=h)$

$ P(J,I=i,H=h) \propto \sum_{L,S,G,H,I,D,C} \phi_J(J,L,S)\phi_L(L,G)\phi_S(S,I)\phi_G(G,I,D)\phi_H(H,G,J)\phi_I(I)\phi_D(C,D)\phi_C(C) $ <= Reduce Factors by Evidence

$ = \sum_{L,S,G,D,C} \phi_J(J,L,S)\phi_L(L,G)\phi_S^{'}(S)\phi_G^{'}(G,D)\phi_H^{'}(G,J)\phi_D(C,D)\phi_C(C) $ <= Eliminate Variables As Before

** Query 2 (Conditional): ** $P(J|I=i,H=h)$

$ P(J|I=i,H=h) = \frac{P(J,I=i,H=h)}{P(I=i,H=h)} $

### C. Elimination Ordering

** a. Complexity **

* Complexity of VE is linear of
    * Size of model
    * Size of largest factor generated
* Size of factor is exponential of its scope

** b. Ordering **

* Start from factors with smaller scopes.

** c. Finding the Best Ordering **

* The problem is NP-Complete, so we use *Heuristic Cost Function* instead: 

    * At each decision point, eliminate node with smallest cost.

* Possible Cost Functions:

    * Min-Neighbors: # neighbors in current graph.
    * Min-Weight: weight (# of values) of factor formed.
    * Min-Fill: Number of new filled edges.
    * Weighted Min-Fill: Total weight of new filled edges (edge weight = product of weights of the 2 nodes).

### D. Moralization: BN $\Rightarrow$ MN

**Method:** Add an edge between pairs of parents involved (A & B in the following example) in all V-structures (NB: V-structures can emerge in the process of VE).

**Induced Graph:** *$X_i$ and $X_j$ are connected if they appeard in the same factor in a run of the VE algorithm (with a certain ordering $\alpha$)*.

* Induced Width: The width of an induced graph is the number of nodes in the largest clique in the graph minus 1.
* Minimal Induced Width: The minimal induced width over all possible VE ordering $\alpha$ on a graph $K$ (best performance).

**Justification:** $\phi(A,C)\phi(B,C)$ in BN $\Rightarrow \phi(A,B,C)$ in MN.

In [ ]:
#  A    B       A----B
#   \_ /_    =>  \ /
#     C           C

### E. Cliques

**Definition: ** The maximal fully-connected subgraph involving a set of variables $\{X_i,...,X_k\}$.

**Theorem a: ** Every factor produced during VE is a clique in the induced graph.

**Theorem b: ** Every (maximal) clique in the induced graph is a factor produced during VE.